# Phân lớp ảnh dùng Keras với VGG16 (pretrained-model)

Ví dụ sau minh hoạ một ứng dụng phân lớp ảnh (image classification).

Ứng dụng sử dụng thư viện Keras, với bộ phân lớp (classifier) đã được huấn luyện sẵn 
dùng mạng neural VGG16. Bộ dữ liệu dùng để huấn luyện là ImageNet-1K - có 1,000 nhãn (label).

Kết quả trả về là xác suất (prob) của từng nhãn - xác suất càng lớn - khả năng nhãn được gán càng cao.

Một demo tương tự có thể xem tại website: http://goliath.liacs.nl/

In [ ]:
# written by B2DL 
# adapted from https://machinelearningmastery.com/use-pre-trained-vgg-model-classify-objects-photographs/
from keras.preprocessing.image import load_img
from keras.preprocessing.image import img_to_array
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from keras.applications.vgg16 import VGG16

from skimage import  io
import cv2
import matplotlib.pyplot as plt


# load image from URL - dùng skimage (opencv không hỗ trợ)
#url = 'http://k14.vcmedia.vn/2016/5-1461127393310.jpg'
url = 'http://farm1.static.flickr.com/116/265424705_d835d8dacb.jpg'
img = io.imread(url)
img_name = 'test-vgg16-keras.jpg'
cv2.imwrite(img_name, cv2.cvtColor(img, cv2.COLOR_RGB2BGR)) 

# đọc ảnh từ file
img = cv2.imread(img_name)

plt.axis('off')
plt.title(img_name)
plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))
plt.show()

# load the model 
# The first time you run this example, 
# Keras will download the weight files from the Internet 
# and store them in the ~/.keras/models directory.
model = VGG16()

# load an image from file
image = load_img(img_name, target_size=(224, 224))

# convert the image pixels to a numpy array
image = img_to_array(image)

# reshape data for the model 
# The network expects one or more images as input; 
# that means the input array will need to be 4-dimensional: 
# samples, rows, columns, and channels.
image = image.reshape((1, image.shape[0], image.shape[1], image.shape[2]))

# prepare the image for the VGG model
# the only preprocessing we do is subtracting the mean RGB value, 
# computed on the training set, from each pixel.
# Keras provides a function called preprocess_input() to prepare new input for the network.
image = preprocess_input(image)

# predict the probability across all output classes
# call the predict() function on the model 
# in order to get a prediction of the probability 
# of the image belonging to each of the 1000 known object types.
yhat = model.predict(image)

# convert the probabilities to class labels
label = decode_predictions(yhat, top=5)[0]
print('Predicted (labelID, labelName, prob):', label)

# print deep model
# print(model.summary())


In [ ]:
'''Trains a simple convnet on the MNIST dataset.
Gets to 99.25% test accuracy after 12 epochs
(there is still a lot of margin for parameter tuning).
16 seconds per epoch on a GRID K520 GPU.
'''

from __future__ import print_function
import keras
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from keras import backend as K

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

if K.image_data_format() == 'channels_first':
    x_train = x_train.reshape(x_train.shape[0], 1, img_rows, img_cols)
    x_test = x_test.reshape(x_test.shape[0], 1, img_rows, img_cols)
    input_shape = (1, img_rows, img_cols)
else:
    x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
    x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
    input_shape = (img_rows, img_cols, 1)

x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)

model = Sequential()
model.add(Conv2D(32, kernel_size=(3, 3),
                 activation='relu',
                 input_shape=input_shape))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])